In [1]:
pip install langchain

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 15.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 15.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 13.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install langchain langchain-community openai

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------------------- ------------------- 1.3/2.5 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/683.3 kB ? eta -:--:--
   ---------------------------------------- 683.3/683.3 kB 9.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# 기본 데이터 처리 및 입출력
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

# LangChain 및 OpenAI API 관련
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI

import os

In [ ]:
os.environ["OPENAI_API_KEY"] = # api 키

In [6]:
# tqdm 초기화:
tqdm.pandas()

In [29]:
# 다수결 방식으로 라벨링을 수행하는 함수: 5번 시행중 3개 이상 일치하는 라벨을 선택
def classify_scandal_labels_multi(text):
    from collections import Counter

    results = []
    for _ in range(5):
        labels = classify_scandal_labels(text)
        results.append(tuple(labels))  # 튜플로 처리하여 Counter에서 집계

    # 다수결 방식으로 상위 3개 선택
    flat_labels = [label for labels in results for label in labels if pd.notna(label)]
    common_labels = [label for label, _ in Counter(flat_labels).most_common(3)]

    while len(common_labels) < 3:
        common_labels.append(np.nan)

    return common_labels[:3]

In [7]:
# 불용어 처리 후 재 라벨링
# 1. CSV 로드
df = pd.read_csv("C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/스캔들/ALL논란_Final.csv")

In [24]:
# 1.1 불용어 처리: 사건 내용 정제 + 불용어 제거
df.head(3)

# 조사
particles_to_remove = {
    "은", "는", "이", "가", "을", "를", "에", "의", "도", "과", "와",
    "에서", "부터", "까지", "만", "든", "로", "으로", "라도", "조차"
}

# 한 글자지만 의미가 중요한 단어
important_single_chars = {"팬", "군", "죄", "법", "술", "약"}

with open('korean_stopwords.txt', encoding='utf-8') as f:
    stopwords = set(w.strip() for w in f.readlines() if w.strip())

def clean_text(text):
    if pd.isnull(text):
        return ""
    
    # 0. 날짜 표현 제거
    text = re.sub(r"\d{4}년\s*\d{1,2}월\s*\d{1,2}일", " ", text)
    text = re.sub(r"\d{4}년", " ", text)
    text = re.sub(r"\d{1,2}월", " ", text)
    text = re.sub(r"\d{1,2}일", " ", text)

    # 조사에 대한 조건 추가
    # 1. 명사 + 조사 형태 제거 (예: 정국은 → 정국)
    text = re.sub(r"([가-힣]+)(으로|에서|부터|까지|라도|조차|은|는|이|가|을|를)\b", r"\1", text)
    # 과 -> 취재결과 경우로 생략
    
    # 1. 특수문자 및 한글/숫자/공백 이외 제거
    text = re.sub(r"[^가-힣0-9\s]", " ", text)
    
    # 2. 중복 공백 제거 및 좌우 공백 제거
    text = re.sub(r"\s+", " ", text).strip()
    
    # 3. 불용어 목록에 있는 단어 제거
    tokens = []
    for t in text.split():
        if t in stopwords:
            continue
        if t in particles_to_remove:
            continue
        if len(t) == 1 and t not in important_single_chars:
            continue
        tokens.append(t)
    
    return " ".join(tokens)

df['clean_text'] = df['사건 내용'].apply(clean_text)

In [25]:
df.head(3)

,소속사,그룹,연예인 이름,사건 날짜,사건 내용,clean_text
0,HYBE,방탄소년단,지민,2022-04-25,"2022년 4월 25일, 언론사 비즈한국의 취재결과 건강보험료 체납으로 보유하고 있...",언론사 비즈한국의 취재결과 건강보험료 체납 보유하고 있던 아파트 압류당한 사실 뒤늦...
1,HYBE,방탄소년단,정국,2024-01-06,"2024년 1월 6일, 정국이 입대 후 신병 교육을 받는 동안 발생했다. 탈취범은 ...",정국 입대 신병 교육 발생했다 탈취범 정국의 동의 정국 명의로 3개의 증권 계좌 무...
2,HYBE,엔하이픈,정원,2021-11-18,"2021년 11월 18일 오후 5시 , ENHYPEN 은 D-1 이라는 제목으로 V...",오후 5시 이라 제목 진행하였다 자리 개최되 번째 팬미팅에 소식 전했다 그렇게 팬들...


In [26]:
# 2. 라벨링 기준 정의
scan_labels = {
    "병역 문제": "군 입대, 대체복무 병역 면제 논란",
    "범죄 혐의": "마약, 음주운전, 폭행, 협박 등",
    "세금 문제": "세금 누락, 탈세, 투자 비리 등",
    "성 관련": "성희롱, 성폭력, 사생활 유포 등",
    "사생활": "열애설, 혼전 임신, 연애 등",
    "팬 대응": "팬 무시, 사생 대응 발언 등",
    "발언 문제": "SNS 발언, 부적절 표현",
    "사회적 감수성": "성차별, 인종차별 발언 등",
    "종교/이념": "정치/종교적 편향 발언",
    "혐의정보 유포": "사이버 명예훼손/혐의 영상 유포",
    "무의식적 태도": "방송 중 실언, 무례함 등",
    "기타": "분류 어려운 사건 또는 해프닝",
    "무혐의": "혐의 없음, 무죄 판결"
}

label_criteria = "\n".join([f"{k}: {v}" for k, v in scan_labels.items()])
label_names = ", ".join(scan_labels.keys())

# 3. LangChain 프롬프트 구성 (chat_history 제거 버전)
prompt_template = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template(
         """
당신은 한국 연예인의 다양한 이슈 및 사건을 특정 카테고리로 분류하는 분류 전문가입니다.

이슈란 다음과 같은 모든 내용을 포함합니다:
사건, 논란, 의혹, 발표, 활동, 해명, 논평, 입대, 제대, 결혼 발표 등 사회적으로 주목받은 일들입니다.
단순 사실이나 발표도 해당 카테고리에 포함시켜 주세요. 논란만을 분류 대상으로 보지 마세요.

아래는 당신이 사용할 라벨 목록과 각 라벨의 기준입니다:

{label_criteria}

---

기사 본문:
"{text}"

---

위 기사를 읽고 아래 라벨 목록 중에서 가장 관련 있는 라벨을 최대 3개까지 콤마(,)로 구분하여 출력하세요.

- 꼭 관련된 라벨만 골라야 하며, 라벨명 외에는 아무것도 출력하지 마세요.
- 하나도 해당되지 않는 경우에만 'nan' 또는 'None' 중 하나로만 답하세요. 애매하거나 확신이 없다면 라벨을 선택하지마세요.

사용 가능한 라벨 목록:
{label_names}
        """
    )
])

# 4. LLMChain 구성
llm_chain = LLMChain(
    llm=ChatOpenAI(
        temperature=0,
        max_tokens=512,
        model_name="gpt-4o"
    ),
    prompt=prompt_template
)

# 5. 라벨 추출 함수
def classify_scandal_labels(text):
    try:
        response = llm_chain.invoke({
            "text": text,
            "label_criteria": label_criteria,
            "label_names": label_names
        })
        result = response['text'].strip()

        # nan 처리
        if result.lower() in ['none', 'nan', '해당 없음']:
            return [np.nan, np.nan, np.nan]

        # 유효한 라벨만 추출 (최대 3개)
        labels = [l.strip() for l in result.split(",") if l.strip() in scan_labels]
        labels = labels[:3]
        while len(labels) < 3:
            labels.append(np.nan)
        return labels
    except Exception as e:
        print(f"오류 발생: {e}")
        return [np.nan, np.nan, np.nan]

# 6. 라벨링 실행
tqdm.pandas()
df[['라벨 1', '라벨 2', '라벨 3']] = df['clean_text'].progress_apply(classify_scandal_labels).apply(pd.Series)

# 7. 결과 저장
df.to_csv("Fianl.csv", index=False)

###############
# 누락된 라벨 재 라벨링
###############
df = pd.read_csv("ALL논란_Final+라벨링.csv")

# 1. 모든 라벨이 비어 있는 행만 필터링
mask_missing = df[['라벨 1', '라벨 2', '라벨 3']].isnull().all(axis=1)
df_missing = df[mask_missing].copy()

# 2. index 정렬 보존용 열 추가
df_missing['__original_index__'] = df_missing.index

# 3. 라벨링 재시도 (5회 다수결)
df_missing[['라벨 1', '라벨 2', '라벨 3']] = df_missing['clean_text'].progress_apply(classify_scandal_labels_multi).apply(pd.Series)

# 4. index 복원 후 원본 df에 반영
for idx, row in df_missing.iterrows():
    df.loc[row['__original_index__'], ['라벨 1', '라벨 2', '라벨 3']] = row[['라벨 1', '라벨 2', '라벨 3']]

# 5. 저장
df.to_csv("ALL논란_Final+라벨링.csv", index=False)

C:\Users\chica\AppData\Local\Temp\ipykernel_19548\2925259079.py:55: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(
C:\Users\chica\AppData\Local\Temp\ipykernel_19548\2925259079.py:54: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
100%|██████████| 363/363 [09:08<00:00,  1.51s/it]


FileNotFoundError: [Errno 2] No such file or directory: 'ALL논란_Final+라벨링.csv'

In [30]:
###############
# 누락된 라벨 재 라벨링
###############
df = pd.read_csv("Fianl.csv")

# 1. 모든 라벨이 비어 있는 행만 필터링
mask_missing = df[['라벨 1', '라벨 2', '라벨 3']].isnull().all(axis=1)
df_missing = df[mask_missing].copy()

# 2. index 정렬 보존용 열 추가
df_missing['__original_index__'] = df_missing.index

# 3. 라벨링 재시도 (5회 다수결)
df_missing[['라벨 1', '라벨 2', '라벨 3']] = df_missing['clean_text'].progress_apply(classify_scandal_labels_multi).apply(pd.Series)

# 4. index 복원 후 원본 df에 반영
for idx, row in df_missing.iterrows():
    df.loc[row['__original_index__'], ['라벨 1', '라벨 2', '라벨 3']] = row[['라벨 1', '라벨 2', '라벨 3']]

# 5. 저장
df.to_csv("ALL논란_Final+라벨링.csv", index=False)

100%|██████████| 97/97 [10:41<00:00,  6.61s/it]


In [31]:
# 모든 행이 비어 있는 경우
mask_all_nan = df[['라벨 1', '라벨 2', '라벨 3']].isnull().all(axis=1)
df_all_nan = df[mask_all_nan]
print(f"모든 라벨이 비어 있는 행 수: {len(df_all_nan)}")

모든 라벨이 비어 있는 행 수: 87


In [32]:
# 하나 이상의 라벨이 비어 있는 경우
mask_any_nan = df[['라벨 1', '라벨 2', '라벨 3']].isnull().any(axis=1)
df_any_nan = df[mask_any_nan]
print(f"하나 이상 라벨이 비어 있는 행 수: {len(df_any_nan)}")

하나 이상 라벨이 비어 있는 행 수: 307
